In [15]:
import pyvisa
from qcodes.instrument_drivers.rohde_schwarz import (
    RohdeSchwarzZNB8,
    RohdeSchwarzZNBChannel,
)
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.animation as animation

%matplotlib qt

import numpy as np
from numpy import *
from scipy.fft import fft, ifft, fftshift
from scipy.io import savemat
from datetime import datetime, timedelta
import time

import os
import scipy.io as sio
import h5py

import scipy.fft
Global_data_adress = r"C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA"
exp_name = 'Daffing'
adress = Global_data_adress+'\\'+exp_name
if not os.path.exists(adress):
  os.mkdir(adress)
  print("Folder %s created!" % adress)
else:
  print("Folder %s already exists" % adress)

Folder C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA\Daffing already exists


In [16]:
now = datetime.now()
path = adress +'\\'+ '3D_Daffing'
if not os.path.exists(path):
  os.mkdir(path)
  print("Folder %s created!" % path)
else:
  print("Folder %s already exists" % path)

Folder C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA\Daffing\3D_Daffing already exists


In [17]:
def magtodb(mag): return 20*np.log10(mag)

def vna_setup_CW(cw_frequency=6e9, vna_power=-30, bandwidth=10):
    try: print(vna.query('*IDN?'))                 # check if connection already exist
    except:                                        # if no connection
        rm = pyvisa.ResourceManager()
        vna = rm.open_resource('TCPIP0::169.254.36.111::inst0::INSTR') # GPIB connection
        print(vna.query('*IDN?'))
        
    vna.write(':SENS1:BAND {}'.format(str(bandwidth))) # set defoult bandwidth
    vna.write('SOUR1:POW {}'.format(str(vna_power)))   # set defoult power
    vna.write('INIT1:CONT OFF')                        # single-sweep mode
    vna.write('SENS1:SWE:POIN 201')                    # set maximum possible number of points for single sweep
    vna.write(':SENS1:SWE:TYPE CW')                    # set CW mode of measurements
    return vna


def vna_on(): vna.write('OUTP ON')
def vna_off(): vna.write('OUTP OFF')
    
def vna_meas_points(): 
    return int(vna.query('SENS1:SWE:POIN?'))  # should be 201

def vna_sweep_time(): 
    return float(vna.query('SENS1:SWE:TIME?'))  # estimated sweep time for current settings of vna

def vna_set_cw_freq(freq): 
    vna.write('SENS1:FREQ:CW {}'.format(str(freq))) # set single frequency point for CW mode in Hz

def vna_set_power(power): 
    vna.write('SOUR1:POW {}'.format(str(power))) # set power
    
def vna_set_band(bandwidth): 
    vna.write(':SENS1:BAND {}'.format(str(bandwidth))) # set power
    
def vna_meas_cw():
    vna_on()
    vna.write("INIT1:IMM")                              # start measurement
    time.sleep(0.2+float(vna.query('SENS1:SWE:TIME?'))) # this part is important becouse the measurement nedds to finnish before readout process
    vna.write("CALC1:FORM MLOG")                        # format of data
    data_str = vna.query("CALC1:DATA? SDAT")            # readout
    
    data = np.array(data_str.rstrip().split(",")).astype("float64")
    s = data[0::2] + 1j * data[1::2]
    i = data[0::2]
    q = data[1::2]
    return abs(s), angle(s)

In [18]:
rm = pyvisa.ResourceManager()
vna = rm.open_resource('TCPIP0::169.254.36.111::inst0::INSTR') # GPIB connection
vna.write('INIT1:CONT OFF')
vna.write(':SENS1:SWE:TYPE CW')  

20

# TWPA ON

In [19]:
rm = pyvisa.ResourceManager()
anapico = rm.open_resource('TCPIP0::169.254.5.91::inst0::INSTR') # LAN connection
print(anapico.query('*IDN?'))
anapico.write(r':SYST:COMM:LAN:RTMO 1')
freq = 11.252e9
flux_power = -6
anapico.write(r'SOUR1:FREQ {}'.format(str(freq)))
anapico.write(r'SOUR1:POW {}'.format(str(power)))
anapico.write(r'OUTP1 ON')

dc_source = rm.open_resource('GPIB0::3::INSTR')
dc_source.write('VOLT:OFFS {}'.format(str(2.25))) 

16

In [23]:
anapico.close()

In [20]:
nop_fr = 60
nop_mea = 10000
repeat = 100
xs = np.zeros((nop_fr, nop_mea*repeat))
ys = np.zeros((nop_fr, nop_mea*repeat))


freq = np.linspace(5.78095e9, 5.781487e9,nop_fr)

vna.write('SENS1:SWE:POIN {}'.format(nop_mea))
vna.write('SENS1:AVER:STAT OFF')

power = -7
bandwidth = 1000000
vna_set_power(power)
vna_set_band(bandwidth)

now = datetime.now()
print('Start: ', now.strftime(r'%H-%M-%S'))
runtime = 11.2*nop_fr*repeat/10000*nop_mea
print('Will stop at ', str((now+timedelta(seconds=runtime)).strftime(r'%d/%m %H-%M-%S')))

Start:  15-12-26
Will stop at  02/11 09-52-26


In [21]:
def get_mea(j):
    Mag = np.array([])
    Pha = np.array([])
    for k in range(repeat):
        vna_set_cw_freq(freq[j])
        Mag_1, Pha_1 = vna_meas_cw()
        Mag = np.append(Mag, Mag_1)
        Pha = np.append(Pha, Pha_1)
        
#     now = datetime.now()
#     print(now.strftime(r'%H-%M-%S'), ' run:', j)
    
    return Mag, Pha
        

dict_data=dict()
dict_data['freq'] = freq
dict_data['power'] = power
dict_data['vna_bandwidth'] = bandwidth
experiment_name = '3D_Hist_1freq_dir_order'
file_name = path+'\\'+experiment_name+'.mat'


for j in range(nop_fr):
    Mag, Pha = get_mea(j)
    xs[j] = Mag
    ys[j] = Pha
    dict_data['mag_{}'.format(j)] = np.array(Mag)
    dict_data['pha_{}'.format(j)] = np.array(Pha)
    savemat(file_name, dict_data)
    print(' run: {}/{}'.format(j, len(freq)))

now = datetime.now()    
print('Stop: ', now.strftime(r'%H-%M-%S'))

 run: 0/60
 run: 1/60
 run: 2/60
 run: 3/60
 run: 4/60
 run: 5/60
 run: 6/60
 run: 7/60
 run: 8/60
 run: 9/60
 run: 10/60
 run: 11/60
 run: 12/60
 run: 13/60
 run: 14/60
 run: 15/60
 run: 16/60
 run: 17/60
 run: 18/60
 run: 19/60
 run: 20/60
 run: 21/60
 run: 22/60
 run: 23/60
 run: 24/60
 run: 25/60
 run: 26/60
 run: 27/60
 run: 28/60
 run: 29/60
 run: 30/60
 run: 31/60
 run: 32/60
 run: 33/60
 run: 34/60
 run: 35/60
 run: 36/60
 run: 37/60
 run: 38/60
 run: 39/60
 run: 40/60
 run: 41/60
 run: 42/60
 run: 43/60
 run: 44/60
 run: 45/60
 run: 46/60
 run: 47/60
 run: 48/60
 run: 49/60
 run: 50/60
 run: 51/60
 run: 52/60
 run: 53/60
 run: 54/60
 run: 55/60
 run: 56/60
 run: 57/60
 run: 58/60
 run: 59/60
Stop:  09-47-26


In [22]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.title('Pow=-7, Band = 1MHz, direct sweep, MAG')

# HIST_BINS_mag = np.linspace(1, 3, 5000)
# HIST_BINS_pha = np.linspace(-3.5, -1.6, 5000)

HIST_BINS_mag = np.linspace(np.min(xs), np.max(xs), 5000)
HIST_BINS_pha = np.linspace(np.min(ys), np.max(ys), 5000)

x = (HIST_BINS_mag[1:]+HIST_BINS_mag[:-1])/2
hi = np.zeros((nop_fr, len(x)))


for j in range(len(freq)):
    
    hi[j], bins = np.histogram(xs[j], bins=HIST_BINS_mag)
    
    
cmap = plt.get_cmap('PuBu_r').copy()
cmap.set_extremes(under='#4984b8')
    
boundaries = np.arange(0, 100, 200)    
c = ax.pcolor(x, freq, hi, cmap=cmap, 
               vmin=1)
fig.colorbar(c, ax=ax)

ax.set_xlabel('X')
ax.set_ylabel('Y')

plt.show()

fig = plt.figure()
ax = fig.add_subplot(111)
plt.title('Pow=-7, Band = 10kHz, dual sweep, PHA')

x = (HIST_BINS_pha[1:]+HIST_BINS_pha[:-1])/2
hi = np.zeros((nop_fr, len(x)))

for j in range(len(freq)):
    
    hi[j], bins = np.histogram(ys[j], bins=HIST_BINS_pha)
    
    
c = ax.pcolor(x, freq, hi, cmap=cmap, vmin=1)
fig.colorbar(c, ax=ax)

ax.set_xlabel('X')
ax.set_ylabel('Y')

plt.show()

In [65]:
dict_data=dict()
dict_data['freq'] = freq
dict_data['power'] = power
dict_data['vna_bandwidth'] = bandwidth

dict_data['mag_s'] = np.array(xs)
dict_data['pha_s'] = np.array(ys)

experiment_name = 'S2_3D_Hist_110freq_inv_order'
file_name = path+'\\'+experiment_name+'_'+now.strftime(r'%H-%M-%S')+'.mat'
savemat(file_name, dict_data)

In [21]:
initial_folder = path

folders = os.listdir(initial_folder)
for i in range(len(folders)):
    if ".png" in folders[i]: pass
    else: 
        print(r"[%d]" %(i), '----------->', folders[i]) 

[0] -----------> 3D_Hist_1freq_dir_order_test.mat


In [180]:
file = folders[12]
mat = sio.loadmat(initial_folder+"\\"+file)


vna_bandwidth = mat['vna_bandwidth']
vna_powers = mat['power']
freq = mat['freq'][0]

mag_dir = mat['mag_s']
pha_dir = mat['pha_s']


file = folders[13]
mat = sio.loadmat(initial_folder+"\\"+file)


vna_bandwidth = mat['vna_bandwidth']
vna_powers = mat['power']
freq = mat['freq'][0]

mag_inv = mat['mag_s']
pha_inv = mat['pha_s']

# mat

In [181]:
xs = mag_dir
ys = pha_dir

In [178]:
xs = np.zeros((len(freq), len(mag_dir[0])*2))
ys = np.zeros((len(freq), len(mag_dir[0])*2))

xs[:,:len(mag_dir[0])] = mag_dir
ys[:,:len(mag_dir[0])] = pha_dir

for j in range(len(freq)):
    ys[j,len(mag_dir[0]):] = pha_inv[len(freq)-j-1]
    xs[j,len(mag_dir[0]):] = mag_inv[len(freq)-j-1]

    
shape(xs)

(110, 2000000)

In [173]:
def fft_f(bandwidth=10000, nop=len(xs[0])):
    f = np.linspace(-bandwidth/2, bandwidth/2, nop)
    return f

fig = plt.figure()
ax = fig.add_subplot(111)
for j in array([0, 50, 109], dtype=int):
    F = abs(scipy.fft.fft(xs[j]-mean(xs[j])))**2
    ax.loglog(fft_f(), F)

In [11]:
fig = plt.figure()
ax = fig.add_subplot(111)

for j in array([0], dtype=int):
    ax.plot(xs[j])